In [182]:
%pylab inline
# -*- coding: utf-8 -*-

Populating the interactive namespace from numpy and matplotlib


In [205]:
import re
import itertools
import xml.etree.ElementTree as ET

In [34]:
tropnames = {'etnakhta': u'\u0591', 'segol': u'\u0592', 'shalshelet': u'\u0593', 'katan': u'\u0594', 'gadol': u'\u0595',
             'tipkha': u'\u0596', 'revii': u'\u0597', 'tsinnorit': u'\u0598', 'pashta': u'\u0599', 'yetiv': u'\u059a', 'tevir': u'\u059b',
             'geresh': u'\u059c', 'gereshmukdam': u'\u059d', 'gershayim': u'\u059e', 'karnepara': u'\u059f', 'telishagedola': u'\u05a0',
             'pazer': u'\u05a1', 'munakh': u'\u05a3', 'mapakh': u'\u05a4', 'merkha': u'\u05a5',
             'merkhakfula': u'\u05a6', 'darga': u'\u05a7', 'kadma': u'\u05a8', 'telishaketana': u'\u05a9', 'yerakhbenyomo': u'\u05aa',
             'sofpasuk': u'\u05c3', 'zarka': u'\u05ae'}

# per wikipedia: Note that both marks have been wrongly named by Unicode.[5][6] Zarqa/tsinnor corresponds to Unicode
# "Hebrew accent zinor", code point U+05AE (where "zinor" is a misspelled form for tsinnor), while tsinnorit maps to
# "Hebrew accent zarqa", code point U+0598. 

tropchars = {tropnames[t]: t for t in tropnames}

In [160]:
sfarim = ['bereshit', 'shmot', 'vayikra', 'bmidbar', 'dvarim']

tropstrings = {}
for sefer in sfarim:
    tropstrings[sefer] = {}
    tree = ET.parse(sefer + '.xml')
    root = tree.getroot() 
    prakim = root.findall('.//c')
    for perek in prakim:
        pereknum = int(perek.attrib['n'])
        if pereknum not in tropstrings[sefer]: tropstrings[sefer][pereknum] = {}
        psukim = perek.findall('v')
        for pasuk in psukim:
            pasuknum = int(pasuk.attrib['n'])
            if pasuknum not in tropstrings[sefer][pereknum]:
                # row = {name: 0 for name in tropnames.keys()}
                # row['sefer'] = sefer
                # row['pasuk'] = pasuknum
                # row['perek'] = pereknum
                tropstrings[sefer][pereknum][pasuknum] = ''
            words = pasuk.findall('w')
            wordslist = map(lambda w: list(w.text), words)
            troplist = map(lambda w: filter(lambda c: c in tropchars, w), wordslist)
            troplist = filter(lambda w: len(w) > 0, troplist)
            # there's probably a better way to do this, but I can't think of it
            flattroplist = []
            for w in troplist:
                for c in w:
                    flattroplist.append(c)
            tropstrings[sefer][pereknum][pasuknum] += ''.join(flattroplist)

In [197]:
# there's probably a smart way to add these things while going through the first time (above), but whatever.

munakhrevii = 0
for sefer in tropstrings:
    for perek in tropstrings[sefer]:
        for pasuk in tropstrings[sefer][perek]:
#             print psukim[pasuk]
            found = re.findall('^'+tropnames['munakh'] + tropnames['revii'], tropstrings[sefer][perek][pasuk])
            munakhrevii += len(found)

In [ ]:
sequencetree = {}

It's not a true single tree because each root trop is a seperate tree; there's no single parent. So it seems legit to iterate through the tropnames to start.

The terminating condition is when count on a branch goes to 0. This will always happen once something is added after a *sof pasuk*.

I think in order to keep track of what my sequence is, it makes the most sense to do this depth first.

In [198]:
def getcounts(seq):
    seqcount = 0
    for sefer in tropstrings:
        for perek in tropstrings[sefer]:
            for pasuk in tropstrings[sefer][perek]:
    #             print psukim[pasuk]
                found = re.findall('^'+tropnames['munakh'] + tropnames['revii'], tropstrings[sefer][perek][pasuk])
                seqcount += len(found)
    return seqcount

390

In [208]:
tropperms = itertools.permutations(tropnames)